In [1]:
# Check the project workspace and set paths for folders.

import os
os.chdir('E:\DJP\Data')  #<---- Change Directory accordingly
path = r'.\DJP\Data' #<------ Change Path accordingly
print("Project Workspace:", os.getcwd())
print()

# Import all necessary packages for the project.
import pandas as pd
import statistics as myStats
import glob
import numpy as np
from pandasql import sqldf
from pandas.tseries.offsets import MonthEnd

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

pd.options.mode.chained_assignment = None  # default='warn'


Project Workspace: E:\DJP\Data



In [2]:
# Load files.

EM = pd.read_csv('Employer master.csv')
PT = pd.read_csv('Payment transactions.csv')

Year_Start_Date = '2018-01-01'
Year_End_Date = '2018-12-31'

In [3]:
# Check Employee Master dataset.
EM

# Converting date "9999-12-31" to "2262-01-01" because pandas do not support this date range and will not impact our metrics.
EM.loc[EM.Effective_To == '9999-12-31', 'Effective_To'] = '2262-01-01'
EM

,Employer_No,Effective_From,Effective_To,Status,Tier
0,100269,2002-06-29,2016-05-23,Open,3
1,100269,2016-05-24,2018-11-02,Open,3
2,100269,2018-11-03,2018-11-04,Open,3
3,100269,2018-11-05,2018-11-13,Open,3
4,100269,2018-11-14,2018-11-21,Open,3
...,...,...,...,...,...
5781,100290,2018-02-01,2262-01-01,Open,4
5782,100761,2018-03-19,2262-01-01,Open,4
5783,100493,2018-06-29,2262-01-01,Open,4
5784,100621,2018-05-18,2262-01-01,Open,3


In [4]:
# Check Payment Transactions Dataset
PT

,Employer_No,Cash_Received_Date,Total_Amt
0,100951,2018-08-06,2000.00
1,100029,2018-09-26,1539.47
2,100807,2018-10-15,430.00
3,100130,2019-01-03,942.05
4,100147,2018-11-06,525.70
...,...,...,...
14848,100951,2018-06-04,2000.00
14849,100526,2017-12-21,1032.27
14850,100526,2017-12-21,85.95
14851,100184,2018-01-03,1498.08


In [5]:
#Grouping starts here. Group to find "Num payments", "Total_Amt" for all months of 2018.

aggregations1 = {
   "Total_Amt": "sum" 
}

Group1 = PT.groupby(by = ["Employer_No","Cash_Received_Date"], as_index = False).agg(aggregations1).rename(columns = {"Total_Amt": "Total_Amount_of_Payments"})
Group1

,Employer_No,Cash_Received_Date,Total_Amount_of_Payments
0,100001,2018-05-21,121.60
1,100001,2018-06-28,547.20
2,100001,2018-07-09,275.72
3,100001,2018-08-31,394.26
4,100001,2018-09-11,311.13
...,...,...,...
6718,101000,2018-07-06,1102.44
6719,101000,2018-08-24,353.00
6720,101000,2018-10-11,4230.51
6721,101000,2018-11-27,336.75


In [6]:
Group1['Month_End_Date'] = pd.to_datetime(Group1['Cash_Received_Date'], format="%Y-%m") + MonthEnd(0)
Group1

,Employer_No,Cash_Received_Date,Total_Amount_of_Payments,Month_End_Date
0,100001,2018-05-21,121.60,2018-05-31
1,100001,2018-06-28,547.20,2018-06-30
2,100001,2018-07-09,275.72,2018-07-31
3,100001,2018-08-31,394.26,2018-08-31
4,100001,2018-09-11,311.13,2018-09-30
...,...,...,...,...
6718,101000,2018-07-06,1102.44,2018-07-31
6719,101000,2018-08-24,353.00,2018-08-31
6720,101000,2018-10-11,4230.51,2018-10-31
6721,101000,2018-11-27,336.75,2018-11-30


In [7]:
EM['Effective_From'] = EM['Effective_From'].astype('str')
EM['Effective_To'] = EM['Effective_To'].astype('str')

New_Employers_Jan = EM[(EM['Effective_From'] >= '2018-01-01') & (EM['Effective_From'] <= '2018-01-31') & (EM['Status'] == 'Open')]
New_Employers_Jan['Month_End_Date'] = '2018-01-31'

New_Employers_Feb = EM[(EM['Effective_From'] >= '2018-02-01') & (EM['Effective_From'] <= '2018-02-28') & (EM['Status'] == 'Open')]
New_Employers_Feb['Month_End_Date'] = '2018-02-28'

New_Employers_Mar = EM[(EM['Effective_From'] >= '2018-03-01') & (EM['Effective_From'] <= '2018-03-31') & (EM['Status'] == 'Open')]
New_Employers_Mar['Month_End_Date'] = '2018-03-31'

New_Employers_Apr = EM[(EM['Effective_From'] >= '2018-04-01') & (EM['Effective_From'] <= '2018-04-30') & (EM['Status'] == 'Open')]
New_Employers_Apr['Month_End_Date'] = '2018-04-30'

New_Employers_May = EM[(EM['Effective_From'] >= '2018-05-01') & (EM['Effective_From'] <= '2018-05-31') & (EM['Status'] == 'Open')]
New_Employers_May['Month_End_Date'] = '2018-05-31'

New_Employers_Jun = EM[(EM['Effective_From'] >= '2018-06-01') & (EM['Effective_From'] <= '2018-06-30') & (EM['Status'] == 'Open')]
New_Employers_Jun['Month_End_Date'] = '2018-06-30'

New_Employers_Jul = EM[(EM['Effective_From'] >= '2018-07-01') & (EM['Effective_From'] <= '2018-07-31') & (EM['Status'] == 'Open')]
New_Employers_Jul['Month_End_Date'] = '2018-07-31'

New_Employers_Aug = EM[(EM['Effective_From'] >= '2018-08-01') & (EM['Effective_From'] <= '2018-08-31') & (EM['Status'] == 'Open')]
New_Employers_Aug['Month_End_Date'] = '2018-08-31'

New_Employers_Sep = EM[(EM['Effective_From'] >= '2018-09-01') & (EM['Effective_From'] <= '2018-09-30') & (EM['Status'] == 'Open')]
New_Employers_Sep['Month_End_Date'] = '2018-09-30'

New_Employers_Oct = EM[(EM['Effective_From'] >= '2018-10-01') & (EM['Effective_From'] <= '2018-10-31') & (EM['Status'] == 'Open')]
New_Employers_Oct['Month_End_Date'] = '2018-10-31'

New_Employers_Nov = EM[(EM['Effective_From'] >= '2018-11-01') & (EM['Effective_From'] <= '2018-11-30') & (EM['Status'] == 'Open')]
New_Employers_Nov['Month_End_Date'] = '2018-11-30'

New_Employers_Dec = EM[(EM['Effective_From'] >= '2018-12-01') & (EM['Effective_From'] <= '2018-12-31') & (EM['Status'] == 'Open')]
New_Employers_Dec['Month_End_Date'] = '2018-12-31'


In [8]:
aggregations2 = {
   "Employer_No": "count" 
}

GroupNewJan = New_Employers_Jan.groupby(by = ["Tier","Month_End_Date"], as_index = False).agg(aggregations2).rename(columns = {"Employer_No": "New_Employers"})
GroupNewFeb = New_Employers_Feb.groupby(by = ["Tier","Month_End_Date"], as_index = False).agg(aggregations2).rename(columns = {"Employer_No": "New_Employers"})
GroupNewMar = New_Employers_Mar.groupby(by = ["Tier","Month_End_Date"], as_index = False).agg(aggregations2).rename(columns = {"Employer_No": "New_Employers"})
GroupNewApr = New_Employers_Apr.groupby(by = ["Tier","Month_End_Date"], as_index = False).agg(aggregations2).rename(columns = {"Employer_No": "New_Employers"})
GroupNewMay = New_Employers_May.groupby(by = ["Tier","Month_End_Date"], as_index = False).agg(aggregations2).rename(columns = {"Employer_No": "New_Employers"})
GroupNewJun = New_Employers_Jun.groupby(by = ["Tier","Month_End_Date"], as_index = False).agg(aggregations2).rename(columns = {"Employer_No": "New_Employers"})
GroupNewJul = New_Employers_Jul.groupby(by = ["Tier","Month_End_Date"], as_index = False).agg(aggregations2).rename(columns = {"Employer_No": "New_Employers"})
GroupNewAug = New_Employers_Aug.groupby(by = ["Tier","Month_End_Date"], as_index = False).agg(aggregations2).rename(columns = {"Employer_No": "New_Employers"})
GroupNewSep = New_Employers_Sep.groupby(by = ["Tier","Month_End_Date"], as_index = False).agg(aggregations2).rename(columns = {"Employer_No": "New_Employers"})
GroupNewOct = New_Employers_Oct.groupby(by = ["Tier","Month_End_Date"], as_index = False).agg(aggregations2).rename(columns = {"Employer_No": "New_Employers"})
GroupNewNov = New_Employers_Nov.groupby(by = ["Tier","Month_End_Date"], as_index = False).agg(aggregations2).rename(columns = {"Employer_No": "New_Employers"})
GroupNewDec = New_Employers_Dec.groupby(by = ["Tier","Month_End_Date"], as_index = False).agg(aggregations2).rename(columns = {"Employer_No": "New_Employers"})


In [9]:
#list dataframes you want to append
frame = [GroupNewJan, GroupNewFeb, GroupNewMar, GroupNewApr, GroupNewMay, GroupNewJun, GroupNewJul, GroupNewAug, GroupNewSep, GroupNewOct, GroupNewNov, GroupNewDec]

#new dataframe to store append result
myDataFrame1 = pd.DataFrame()

for df in frame:
    myDataFrame1 = myDataFrame1.append(df)
    
myDataFrame1 = myDataFrame1.sort_values(by=['Tier', 'Month_End_Date'])
myDataFrame1 = myDataFrame1.reset_index(drop=True)
myDataFrame1

,Tier,Month_End_Date,New_Employers
0,1,2018-03-31,1
1,1,2018-04-30,2
2,1,2018-06-30,1
3,1,2018-08-31,1
4,1,2018-10-31,2
5,1,2018-11-30,131
6,1,2018-12-31,40
7,2,2018-01-31,1
8,2,2018-03-31,1
9,2,2018-04-30,3


In [10]:
EM

,Employer_No,Effective_From,Effective_To,Status,Tier
0,100269,2002-06-29,2016-05-23,Open,3
1,100269,2016-05-24,2018-11-02,Open,3
2,100269,2018-11-03,2018-11-04,Open,3
3,100269,2018-11-05,2018-11-13,Open,3
4,100269,2018-11-14,2018-11-21,Open,3
...,...,...,...,...,...
5781,100290,2018-02-01,2262-01-01,Open,4
5782,100761,2018-03-19,2262-01-01,Open,4
5783,100493,2018-06-29,2262-01-01,Open,4
5784,100621,2018-05-18,2262-01-01,Open,3


In [11]:
TestEM = EM
TestEM

,Employer_No,Effective_From,Effective_To,Status,Tier
0,100269,2002-06-29,2016-05-23,Open,3
1,100269,2016-05-24,2018-11-02,Open,3
2,100269,2018-11-03,2018-11-04,Open,3
3,100269,2018-11-05,2018-11-13,Open,3
4,100269,2018-11-14,2018-11-21,Open,3
...,...,...,...,...,...
5781,100290,2018-02-01,2262-01-01,Open,4
5782,100761,2018-03-19,2262-01-01,Open,4
5783,100493,2018-06-29,2262-01-01,Open,4
5784,100621,2018-05-18,2262-01-01,Open,3


In [12]:
TestEM['Effective_From'] = TestEM['Effective_From'].astype('str')
TestEM['Effective_To'] = TestEM['Effective_To'].astype('str')

Open_Employers_Jan = TestEM[(TestEM['Effective_To'] >= '2018-01-31') & (TestEM['Status'] == 'Open') & (TestEM['Effective_From'] <= '2018-01-31')]
Open_Employers_Jan['Month_End_Date'] = '2018-01-31'

Open_Employers_Feb = TestEM[(TestEM['Effective_To'] >= '2018-02-28') & (TestEM['Status'] == 'Open') & (TestEM['Effective_From'] <= '2018-02-28')]
Open_Employers_Feb['Month_End_Date'] = '2018-02-28'

Open_Employers_Mar = TestEM[(TestEM['Effective_To'] >= '2018-03-31') & (TestEM['Status'] == 'Open') & (TestEM['Effective_From'] <= '2018-03-31')]
Open_Employers_Mar['Month_End_Date'] = '2018-03-31'

Open_Employers_Apr = TestEM[(TestEM['Effective_To'] >= '2018-04-30') & (TestEM['Status'] == 'Open') & (TestEM['Effective_From'] <= '2018-04-30')]
Open_Employers_Apr['Month_End_Date'] = '2018-04-30'

Open_Employers_May = TestEM[(TestEM['Effective_To'] >= '2018-05-31') & (TestEM['Status'] == 'Open') & (TestEM['Effective_From'] <= '2018-05-31')]
Open_Employers_May['Month_End_Date'] = '2018-05-31'

Open_Employers_Jun = TestEM[(TestEM['Effective_To'] >= '2018-06-30') & (TestEM['Status'] == 'Open') & (TestEM['Effective_From'] <= '2018-06-30')]
Open_Employers_Jun['Month_End_Date'] = '2018-06-30'

Open_Employers_Jul = TestEM[(TestEM['Effective_To'] >= '2018-07-31') & (TestEM['Status'] == 'Open')  & (TestEM['Effective_From'] <= '2018-07-31')]
Open_Employers_Jul['Month_End_Date'] = '2018-07-31'

Open_Employers_Aug = TestEM[(TestEM['Effective_To'] >= '2018-08-31') & (TestEM['Status'] == 'Open')  & (TestEM['Effective_From'] <= '2018-08-31')]
Open_Employers_Aug['Month_End_Date'] = '2018-08-31'

Open_Employers_Sep = TestEM[(TestEM['Effective_To'] >= '2018-09-30') & (TestEM['Status'] == 'Open')  & (TestEM['Effective_From'] <= '2018-09-30')]
Open_Employers_Sep['Month_End_Date'] = '2018-09-30'

Open_Employers_Oct = TestEM[(TestEM['Effective_To'] >= '2018-10-31') & (TestEM['Status'] == 'Open')  & (TestEM['Effective_From'] <= '2018-10-31')]
Open_Employers_Oct['Month_End_Date'] = '2018-10-31'

Open_Employers_Nov = TestEM[(TestEM['Effective_To'] >= '2018-11-30') & (TestEM['Status'] == 'Open')  & (TestEM['Effective_From'] <= '2018-11-31')]
Open_Employers_Nov['Month_End_Date'] = '2018-11-30'

Open_Employers_Dec = TestEM[(TestEM['Effective_To'] >= '2018-12-31') & (TestEM['Status'] == 'Open')  & (TestEM['Effective_From'] <= '2018-12-31')]
Open_Employers_Dec['Month_End_Date'] = '2018-12-31'

In [13]:
aggregations3 = {
   "Employer_No": "count" 
}

GroupOpenJan = Open_Employers_Jan.groupby(by = ["Tier","Month_End_Date"], as_index = False).agg(aggregations3).rename(columns = {"Employer_No": "Open_Employers_at_EOM"})
GroupOpenFeb = Open_Employers_Feb.groupby(by = ["Tier","Month_End_Date"], as_index = False).agg(aggregations3).rename(columns = {"Employer_No": "Open_Employers_at_EOM"})
GroupOpenMar = Open_Employers_Mar.groupby(by = ["Tier","Month_End_Date"], as_index = False).agg(aggregations3).rename(columns = {"Employer_No": "Open_Employers_at_EOM"})
GroupOpenApr = Open_Employers_Apr.groupby(by = ["Tier","Month_End_Date"], as_index = False).agg(aggregations3).rename(columns = {"Employer_No": "Open_Employers_at_EOM"})
GroupOpenMay = Open_Employers_May.groupby(by = ["Tier","Month_End_Date"], as_index = False).agg(aggregations3).rename(columns = {"Employer_No": "Open_Employers_at_EOM"})
GroupOpenJun = Open_Employers_Jun.groupby(by = ["Tier","Month_End_Date"], as_index = False).agg(aggregations3).rename(columns = {"Employer_No": "Open_Employers_at_EOM"})
GroupOpenJul = Open_Employers_Jul.groupby(by = ["Tier","Month_End_Date"], as_index = False).agg(aggregations3).rename(columns = {"Employer_No": "Open_Employers_at_EOM"})
GroupOpenAug = Open_Employers_Aug.groupby(by = ["Tier","Month_End_Date"], as_index = False).agg(aggregations3).rename(columns = {"Employer_No": "Open_Employers_at_EOM"})
GroupOpenSep = Open_Employers_Sep.groupby(by = ["Tier","Month_End_Date"], as_index = False).agg(aggregations3).rename(columns = {"Employer_No": "Open_Employers_at_EOM"})
GroupOpenOct = Open_Employers_Oct.groupby(by = ["Tier","Month_End_Date"], as_index = False).agg(aggregations3).rename(columns = {"Employer_No": "Open_Employers_at_EOM"})
GroupOpenNov = Open_Employers_Nov.groupby(by = ["Tier","Month_End_Date"], as_index = False).agg(aggregations3).rename(columns = {"Employer_No": "Open_Employers_at_EOM"})
GroupOpenDec = Open_Employers_Dec.groupby(by = ["Tier","Month_End_Date"], as_index = False).agg(aggregations3).rename(columns = {"Employer_No": "Open_Employers_at_EOM"})


In [14]:
#list dataframes you want to append
frame2 = [GroupOpenJan, GroupOpenFeb, GroupOpenMar, GroupOpenApr, GroupOpenMay, GroupOpenJun, GroupOpenJul, GroupOpenAug, GroupOpenSep, GroupOpenOct, GroupOpenNov, GroupOpenDec]

#new dataframe to store append result
myDataFrame2 = pd.DataFrame()

for df in frame2:
    myDataFrame2 = myDataFrame2.append(df)
    
myDataFrame2 = myDataFrame2.sort_values(by=['Tier', 'Month_End_Date'])
myDataFrame2 = myDataFrame2.reset_index(drop=True)
myDataFrame2


,Tier,Month_End_Date,Open_Employers_at_EOM
0,1,2018-01-31,47
1,1,2018-02-28,47
2,1,2018-03-31,48
3,1,2018-04-30,50
4,1,2018-05-31,50
5,1,2018-06-30,51
6,1,2018-07-31,51
7,1,2018-08-31,52
8,1,2018-09-30,52
9,1,2018-10-31,53


In [15]:
Merge1 = pd.merge(myDataFrame2,myDataFrame1,on=['Tier','Month_End_Date'], how = 'left')
Merge1['New_Employers'] = Merge1['New_Employers'].fillna(0)
Merge1['New_Employers'] = Merge1['New_Employers'].astype('int')
Merge1

,Tier,Month_End_Date,Open_Employers_at_EOM,New_Employers
0,1,2018-01-31,47,0
1,1,2018-02-28,47,0
2,1,2018-03-31,48,1
3,1,2018-04-30,50,2
4,1,2018-05-31,50,0
5,1,2018-06-30,51,1
6,1,2018-07-31,51,0
7,1,2018-08-31,52,1
8,1,2018-09-30,52,0
9,1,2018-10-31,53,2


In [16]:
Group1

,Employer_No,Cash_Received_Date,Total_Amount_of_Payments,Month_End_Date
0,100001,2018-05-21,121.60,2018-05-31
1,100001,2018-06-28,547.20,2018-06-30
2,100001,2018-07-09,275.72,2018-07-31
3,100001,2018-08-31,394.26,2018-08-31
4,100001,2018-09-11,311.13,2018-09-30
...,...,...,...,...
6718,101000,2018-07-06,1102.44,2018-07-31
6719,101000,2018-08-24,353.00,2018-08-31
6720,101000,2018-10-11,4230.51,2018-10-31
6721,101000,2018-11-27,336.75,2018-11-30


In [17]:
Group1['Cash_Received_Date'] = Group1['Cash_Received_Date'].astype('str')

pysqldf = lambda q: sqldf(q, globals())
q = """SELECT a.Employer_No, a.Cash_Received_Date, a.Total_Amount_of_Payments, a.Month_End_Date, b.Tier
       FROM 
           Group1 a
       LEFT JOIN 
           TestEM b
        ON a.Employer_No = b.Employer_No
        WHERE a.Cash_Received_Date >= Effective_From
        AND a.Cash_Received_Date <= Effective_To;"""

join = pysqldf(q)
joined = pd.DataFrame(join)
joined['Month_End_Date'] = pd.to_datetime(joined['Month_End_Date'])
joined

,Employer_No,Cash_Received_Date,Total_Amount_of_Payments,Month_End_Date,Tier
0,100001,2018-05-21,121.60,2018-05-31,4
1,100001,2018-06-28,547.20,2018-06-30,4
2,100001,2018-07-09,275.72,2018-07-31,4
3,100001,2018-08-31,394.26,2018-08-31,4
4,100001,2018-09-11,311.13,2018-09-30,4
...,...,...,...,...,...
6717,101000,2018-07-06,1102.44,2018-07-31,4
6718,101000,2018-08-24,353.00,2018-08-31,4
6719,101000,2018-10-11,4230.51,2018-10-31,4
6720,101000,2018-11-27,336.75,2018-11-30,4


In [18]:
#Grouping starts here. Group to find "Num payments", "Total_Amt" for all months of 2018.

aggregations4 = {
   "Total_Amount_of_Payments": "sum",
   "Employer_No": "count"
}

myDataFrame3 = joined.groupby(by = ["Tier","Month_End_Date"], as_index = False).agg(aggregations4).rename(columns = {"Total_Amount_of_Payments": "Amount_of_Payments", "Employer_No": "Num_Payments"})
myDataFrame3 = myDataFrame3[(myDataFrame3['Month_End_Date'] >= Year_Start_Date) & (myDataFrame3['Month_End_Date'] <= Year_End_Date)]

myDataFrame3 = myDataFrame3.sort_values(by=['Tier', 'Month_End_Date'])
myDataFrame3 = myDataFrame3.reset_index(drop=True)
myDataFrame3

,Tier,Month_End_Date,Amount_of_Payments,Num_Payments
0,1,2018-01-31,66912.91,25
1,1,2018-02-28,77029.77,22
2,1,2018-04-30,77125.12,23
3,1,2018-05-31,100050.32,28
4,1,2018-06-30,94994.46,25
5,1,2018-07-31,84294.60,24
6,1,2018-08-31,32875.80,17
7,1,2018-09-30,59299.04,19
8,1,2018-10-31,68805.03,26
9,1,2018-11-30,69293.72,15


In [19]:
myDataFrame3.loc[-1] = [1, '2018-03-31', 0.00, 0]  
myDataFrame3.index =  myDataFrame3.index + 1  
myDataFrame3['Month_End_Date'] = pd.to_datetime(myDataFrame3['Month_End_Date'])
myDataFrame4 = myDataFrame3.sort_values(by=['Tier', 'Month_End_Date'])
myDataFrame5 = myDataFrame4.reset_index(drop=True)
myDataFrame5['Month_End_Date'] = myDataFrame5['Month_End_Date'].astype('str')
myDataFrame5

,Tier,Month_End_Date,Amount_of_Payments,Num_Payments
0,1,2018-01-31,66912.91,25
1,1,2018-02-28,77029.77,22
2,1,2018-03-31,0.00,0
3,1,2018-04-30,77125.12,23
4,1,2018-05-31,100050.32,28
5,1,2018-06-30,94994.46,25
6,1,2018-07-31,84294.60,24
7,1,2018-08-31,32875.80,17
8,1,2018-09-30,59299.04,19
9,1,2018-10-31,68805.03,26


In [20]:
Merge2 = pd.merge(myDataFrame5,Merge1,on=['Tier','Month_End_Date'], how = 'inner')
Merge2

,Tier,Month_End_Date,Amount_of_Payments,Num_Payments,Open_Employers_at_EOM,New_Employers
0,1,2018-01-31,66912.91,25,47,0
1,1,2018-02-28,77029.77,22,47,0
2,1,2018-03-31,0.00,0,48,1
3,1,2018-04-30,77125.12,23,50,2
4,1,2018-05-31,100050.32,28,50,0
5,1,2018-06-30,94994.46,25,51,1
6,1,2018-07-31,84294.60,24,51,0
7,1,2018-08-31,32875.80,17,52,1
8,1,2018-09-30,59299.04,19,52,0
9,1,2018-10-31,68805.03,26,53,2


In [21]:
Merge2.to_csv('Final.csv', index=False)